# Augmented answer generate

## Imports

In [1]:
from fastai import *
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import itertools
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from fastai import *
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import pickle

## Load data

In [18]:
df_train = pd.read_csv('data/train.csv')
X_train = df_train.drop(['ID_code', 'target'], axis=1)
y_train = df_train.target

## Train super model

In [6]:
def load_model(model_name):
    with open(model_name, 'rb') as handle:
        model = pickle.load(handle)
    return model    

def make_prediction(model, X):
    return model.predict(X)

def get_all_preds(X, model_names):
    all_preds = {}
    for model_name in model_names:
        i = 0
        models = load_model(model_name)
        for model in models:
            i = i + 1
            y_pred = make_prediction(model, X)
            key = str(model_name) + str(i)
            all_preds[key] = y_pred
            print(f"{key} Done with {len(y_pred)} shape")
    return pd.DataFrame(all_preds)      

In [7]:
model_names = !ls *m
model_names = list(model_names)

In [8]:
X_aug = get_all_preds(X_train, model_names)

adasyn_aug_lgbm_models_cv10.m1 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m2 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m3 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m4 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m5 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m1 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m2 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m3 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m4 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m5 Done with 200000 shape
lgbm_cv10_models.m1 Done with 200000 shape
lgbm_cv10_models.m2 Done with 200000 shape
lgbm_cv10_models.m3 Done with 200000 shape
lgbm_cv10_models.m4 Done with 200000 shape
lgbm_cv10_models.m5 Done with 200000 shape
lgbm_cv10_models.m6 Done with 200000 shape
lgbm_cv10_models.m7 Done with 200000 shape
lgbm_cv10_models.m8 Done with 200000 shape
lgbm_cv10_models.m9 Done with 200000 shape
lgbm_cv10_models.m10 Done with 200000 shape
random_aug_lgbm_models_cv10.m

In [ ]:
#train super model (linear regression) on X_aug and y_train now

In [20]:
X_aug.to_csv('train_augmented.csv', index=False)

### Get preds on test set

In [14]:
df_test = pd.read_csv('data/test.csv')
test_labels = df_test.ID_code
X_test = df_test.drop('ID_code', axis=1)

In [15]:
X_test_aug = get_all_preds(X_test, model_names)

adasyn_aug_lgbm_models_cv10.m1 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m2 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m3 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m4 Done with 200000 shape
adasyn_aug_lgbm_models_cv10.m5 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m1 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m2 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m3 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m4 Done with 200000 shape
bsmote_aug_lgbm_models_cv10.m5 Done with 200000 shape
lgbm_cv10_models.m1 Done with 200000 shape
lgbm_cv10_models.m2 Done with 200000 shape
lgbm_cv10_models.m3 Done with 200000 shape
lgbm_cv10_models.m4 Done with 200000 shape
lgbm_cv10_models.m5 Done with 200000 shape
lgbm_cv10_models.m6 Done with 200000 shape
lgbm_cv10_models.m7 Done with 200000 shape
lgbm_cv10_models.m8 Done with 200000 shape
lgbm_cv10_models.m9 Done with 200000 shape
lgbm_cv10_models.m10 Done with 200000 shape
random_aug_lgbm_models_cv10.m

In [17]:
X_test_aug.to_csv('test_augmented.csv')

# Super-learner

### Simple linear regression

In [62]:
!ls *.csv

answer_1_simple_gbm.csv				      test_augmented.csv
answer_2_lgbm_with_CV10.csv			      train_augmented.csv
answer_3_lgbm_with_augmented_data_smote_and_shiz.csv


In [63]:
augmented_train_x = pd.read_csv('train_augmented.csv')
augmented_y = y_train.copy()

In [64]:
from sklearn import linear_model

In [65]:
reg = linear_model.LinearRegression()
reg.fit(augmented_train_x, augmented_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [66]:
reg.predict(augmented_train_x)

array([-0.04072782,  0.44887592, -0.05197011, ...,  0.16407198,
        0.00194383, -0.05755002])

In [67]:
augmented_test_x = pd.read_csv('test_augmented.csv')
augmented_test_x = augmented_test_x.drop('Unnamed: 0', axis = 1)

In [69]:
psss = reg.predict(augmented_test_x)

In [72]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : psss
})
answer_df.to_csv('answer_4_lgbm_with_augmented_data_smote_and_shiz_and_super_learner.csv', index=False)

### Submit average

In [48]:
augmented_test_x = pd.read_csv('test_augmented.csv')

In [50]:
augmented_test_x = augmented_test_x.drop('Unnamed: 0', axis = 1)

In [60]:
predss = np.average(augmented_test_x, axis=1)

In [55]:
test_labels.shape

(200000,)

# Evaluate all models

In [77]:
from sklearn.model_selection import train_test_split
!ls *csv

answer_1_simple_gbm.csv
answer_2_lgbm_with_CV10.csv
answer_3_lgbm_with_augmented_data_smote_and_shiz.csv
answer_4_lgbm_with_augmented_data_smote_and_shiz_and_super_learner.csv
test_augmented.csv
train_augmented.csv


In [87]:
all_augments = pd.read_csv('train_augmented.csv')
all_augments.head()

,adasyn_aug_lgbm_models_cv10.m1,adasyn_aug_lgbm_models_cv10.m2,adasyn_aug_lgbm_models_cv10.m3,adasyn_aug_lgbm_models_cv10.m4,adasyn_aug_lgbm_models_cv10.m5,bsmote_aug_lgbm_models_cv10.m1,bsmote_aug_lgbm_models_cv10.m2,bsmote_aug_lgbm_models_cv10.m3,bsmote_aug_lgbm_models_cv10.m4,bsmote_aug_lgbm_models_cv10.m5,...,random_aug_lgbm_models_cv10.m1,random_aug_lgbm_models_cv10.m2,random_aug_lgbm_models_cv10.m3,random_aug_lgbm_models_cv10.m4,random_aug_lgbm_models_cv10.m5,smote_aug_lgbm_models_cv10.m1,smote_aug_lgbm_models_cv10.m2,smote_aug_lgbm_models_cv10.m3,smote_aug_lgbm_models_cv10.m4,smote_aug_lgbm_models_cv10.m5
0,0.016623,0.033341,0.037256,0.048084,0.028348,0.007562,0.019911,0.024626,0.029157,0.027314,...,0.020105,0.015250,0.016137,0.018904,0.014611,0.012706,0.039920,0.026462,0.040999,0.035827
1,0.224061,0.370540,0.425310,0.296699,0.280636,0.158633,0.345929,0.362552,0.341628,0.301488,...,0.686547,0.508209,0.502711,0.450762,0.460311,0.206920,0.383714,0.364566,0.383416,0.381592
2,0.097611,0.056676,0.053823,0.091625,0.054415,0.039425,0.037340,0.042293,0.066892,0.057396,...,0.005116,0.005897,0.004836,0.006172,0.005005,0.074559,0.075694,0.063259,0.067911,0.050489
3,0.036070,0.164682,0.159335,0.203573,0.155360,0.074917,0.176158,0.158951,0.184137,0.165549,...,0.551771,0.362405,0.380677,0.429487,0.372332,0.130420,0.175960,0.143898,0.153527,0.129754
4,0.055706,0.150601,0.228857,0.149102,0.123431,0.026243,0.112539,0.150596,0.133850,0.099653,...,0.301496,0.166761,0.235498,0.218351,0.180874,0.045743,0.125034,0.137385,0.158880,0.154681


In [90]:
for column in all_augments:
    if 'random_aug_lgbm' in column:
        y_pred = all_augments[column]
        score = roc_auc_score(y_train, y_pred)
        print(f"Score for column {column} is {score}")

Score for column random_aug_lgbm_models_cv10.m1 is 0.9964923658378732
Score for column random_aug_lgbm_models_cv10.m2 is 0.9968663277458768
Score for column random_aug_lgbm_models_cv10.m3 is 0.9967476952509252
Score for column random_aug_lgbm_models_cv10.m4 is 0.9967630722056556
Score for column random_aug_lgbm_models_cv10.m5 is 0.996846146149656


In [91]:
# Let's try making a prediction only for random wale models

In [92]:
test_augments = pd.read_csv('test_augmented.csv')
test_augments.head()

,adasyn_aug_lgbm_models_cv10.m1,adasyn_aug_lgbm_models_cv10.m2,adasyn_aug_lgbm_models_cv10.m3,adasyn_aug_lgbm_models_cv10.m4,adasyn_aug_lgbm_models_cv10.m5,bsmote_aug_lgbm_models_cv10.m1,bsmote_aug_lgbm_models_cv10.m2,bsmote_aug_lgbm_models_cv10.m3,bsmote_aug_lgbm_models_cv10.m4,bsmote_aug_lgbm_models_cv10.m5,...,random_aug_lgbm_models_cv10.m1,random_aug_lgbm_models_cv10.m2,random_aug_lgbm_models_cv10.m3,random_aug_lgbm_models_cv10.m4,random_aug_lgbm_models_cv10.m5,smote_aug_lgbm_models_cv10.m1,smote_aug_lgbm_models_cv10.m2,smote_aug_lgbm_models_cv10.m3,smote_aug_lgbm_models_cv10.m4,smote_aug_lgbm_models_cv10.m5
0,0.210840,0.423976,0.414336,0.426701,0.382931,0.097990,0.345339,0.366936,0.327532,0.354420,...,0.321245,0.221360,0.295038,0.265720,0.296561,0.358321,0.408347,0.438010,0.429142,0.375794
1,0.248277,0.262594,0.227486,0.272556,0.325919,0.138981,0.259454,0.362740,0.319277,0.338423,...,0.577859,0.482904,0.498362,0.586837,0.542625,0.159322,0.301628,0.304506,0.335108,0.354454
2,0.018197,0.130239,0.115850,0.148812,0.101614,0.005240,0.126774,0.102754,0.103884,0.121339,...,0.484063,0.518016,0.466096,0.434962,0.461928,0.021069,0.091714,0.115162,0.128861,0.110598
3,0.030662,0.198692,0.324841,0.291769,0.255262,0.071924,0.288979,0.287383,0.326068,0.263984,...,0.542375,0.493405,0.571898,0.626264,0.488509,0.028589,0.290549,0.349548,0.214495,0.249461
4,0.096198,0.181725,0.160747,0.236469,0.263639,0.021169,0.107579,0.127188,0.099395,0.141326,...,0.095421,0.089001,0.139223,0.126109,0.113628,0.063415,0.172751,0.214105,0.153486,0.177251


In [101]:
test_augments.columns

Index(['adasyn_aug_lgbm_models_cv10.m1', 'adasyn_aug_lgbm_models_cv10.m2',
       'adasyn_aug_lgbm_models_cv10.m3', 'adasyn_aug_lgbm_models_cv10.m4',
       'adasyn_aug_lgbm_models_cv10.m5', 'bsmote_aug_lgbm_models_cv10.m1',
       'bsmote_aug_lgbm_models_cv10.m2', 'bsmote_aug_lgbm_models_cv10.m3',
       'bsmote_aug_lgbm_models_cv10.m4', 'bsmote_aug_lgbm_models_cv10.m5',
       'lgbm_cv10_models.m1', 'lgbm_cv10_models.m2', 'lgbm_cv10_models.m3',
       'lgbm_cv10_models.m4', 'lgbm_cv10_models.m5', 'lgbm_cv10_models.m6',
       'lgbm_cv10_models.m7', 'lgbm_cv10_models.m8', 'lgbm_cv10_models.m9',
       'lgbm_cv10_models.m10', 'random_aug_lgbm_models_cv10.m1',
       'random_aug_lgbm_models_cv10.m2', 'random_aug_lgbm_models_cv10.m3',
       'random_aug_lgbm_models_cv10.m4', 'random_aug_lgbm_models_cv10.m5',
       'smote_aug_lgbm_models_cv10.m1', 'smote_aug_lgbm_models_cv10.m2',
       'smote_aug_lgbm_models_cv10.m3', 'smote_aug_lgbm_models_cv10.m4',
       'smote_aug_lgbm_models_cv10.m

In [107]:
preds = {}
for column in test_augments:
    if 'random_aug_lgbm' in column:
        preds[column] = test_augments[column]

In [110]:
random_preds = np.average(pd.DataFrame(preds), axis = 1)

In [100]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : random_preds
})
answer_df.to_csv('answer_5_lgbm_with_only_random_augmented_data.csv', index=False)

In [102]:
# Try averaging with old non augmented model

In [103]:
old_answer = pd.read_csv('answer_2_lgbm_with_CV10.csv')

In [105]:
old_answer = old_answer.target

In [114]:
old_answer.head()

0    0.095370
1    0.211337
2    0.181671
3    0.212378
4    0.044031
Name: target, dtype: float64

In [120]:
new_preds = pd.DataFrame(preds)
print(new_preds.shape)
new_preds.head()

(200000, 5)


,random_aug_lgbm_models_cv10.m1,random_aug_lgbm_models_cv10.m2,random_aug_lgbm_models_cv10.m3,random_aug_lgbm_models_cv10.m4,random_aug_lgbm_models_cv10.m5
0,0.321245,0.221360,0.295038,0.265720,0.296561
1,0.577859,0.482904,0.498362,0.586837,0.542625
2,0.484063,0.518016,0.466096,0.434962,0.461928
3,0.542375,0.493405,0.571898,0.626264,0.488509
4,0.095421,0.089001,0.139223,0.126109,0.113628


In [121]:
new_preds['old'] = 10 * old_answer
new_preds.head()

,random_aug_lgbm_models_cv10.m1,random_aug_lgbm_models_cv10.m2,random_aug_lgbm_models_cv10.m3,random_aug_lgbm_models_cv10.m4,random_aug_lgbm_models_cv10.m5,old
0,0.321245,0.221360,0.295038,0.265720,0.296561,0.953704
1,0.577859,0.482904,0.498362,0.586837,0.542625,2.113365
2,0.484063,0.518016,0.466096,0.434962,0.461928,1.816714
3,0.542375,0.493405,0.571898,0.626264,0.488509,2.123776
4,0.095421,0.089001,0.139223,0.126109,0.113628,0.440313


In [122]:
a = new_preds.sum(axis = 1)/15

In [124]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : a
})
answer_df.to_csv('answer_6_lgbm_with_best_and_random_averaged.csv', index=False)